In [1]:
%matplotlib inline
from matplotlib import pyplot as plt 

In [2]:
from glob import glob
import json
import csv
import os

import cv2

import numpy as np

In [3]:
import h5py

In [4]:
DATA_DIR = os.path.join(os.getcwd(), '../data')

In [5]:
RUNS_DIR = ['run1', 'run2']
RUNS_DATA = [os.path.join(DATA_DIR, run_dir) for run_dir in RUNS_DIR]

In [6]:
RUNS_DATA

['/Users/Sumukh/Workspace/CarND/Term1/Projects/CarND-Behavioral-Cloning-P3/notebooks/../data/run1',
 '/Users/Sumukh/Workspace/CarND/Term1/Projects/CarND-Behavioral-Cloning-P3/notebooks/../data/run2']

In [8]:
def read_image(img_path):
    img = cv2.imread(img_path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def flip_image(img):
    return cv2.flip(img, 1)

In [9]:
def preprocess_image(img_path, gray=False, flip=False, crop=False):
    img = read_image(img_path)
    assert img is not None
    img_flip = None
    if gray:
        img = grayscale(img)
        if crop:
            img = img[60: 140, :, :]
    if flip:
        img_flip = flip_image(img)
    return img, img_flip

In [10]:
def preprocess(gray, flip):
    images = []
    angles = []
    for rdir in RUNS_DATA:
        log_file = os.path.join(rdir, 'driving_log.csv')
        with open(log_file, 'r') as _fh:
            log = csv.reader(_fh)
            for line in log:
                measurement = float(line[3])
                
                center, center_flip = preprocess_image(line[0], gray=gray, flip=flip)
                left, left_flip = preprocess_image(line[1], gray=gray, flip=flip)
                right, right_flip = preprocess_image(line[2], gray=gray, flip=flip)

                images.extend([center, left, right])
                angles.extend([measurement]*3)
                if flip:
                    images.extend([center_flip, left_flip, right_flip])
                    angles.extend([-measurement]*3)
    return np.array(images), np.array(angles)

In [13]:
def save(filename, images, angles):
    h5f = h5py.File(os.path.join(DATA_DIR, filename), "w")
    grp = h5f.create_group("data")
    dset1 = grp.create_dataset("images", images.shape, 
                               dtype=np.float32)
    dset2 = grp.create_dataset("angles", angles.shape,
                               dtype=np.float32)
    dset1[...] = images
    dset2[...] = angles
    h5f.close()

In [14]:
flip = False
gray = False

images, angles = preprocess(gray, flip)
save("driving_data.h5", images, angles)

In [15]:
images.shape

(39915, 160, 320, 3)

In [16]:
angles.shape

(39915,)

In [17]:
images[34].shape

(160, 320, 3)

In [18]:
flip = True
gray = True

images, angles = preprocess(gray, flip)
save("driving_data_gf.h5", images, angles)

In [19]:
images.shape

(79830, 160, 320)

In [20]:
angles.shape

(79830,)

In [21]:
flip = True
gray = False

images, angles = preprocess(gray, flip)
save("driving_data_f.h5", images, angles)

OSError: Can't prepare for writing data (file write failed: time = Sun Nov 26 01:38:21 2017
, filename = '/Users/Sumukh/Workspace/CarND/Term1/Projects/CarND-Behavioral-Cloning-P3/notebooks/../data/driving_data_f.h5', file descriptor = 34, errno = 28, error message = 'No space left on device', buf = 0x10a2fe008, total write size = 1048576, bytes this sub-write = 1048576, bytes actually written = 18446744073709551615, offset = 36251372176)